# Qwen Distillation Demo

This notebook consolidates the core code from `test7` and walks through:
1. Fetching K-line data from EastMoney.
2. Building a simple prompt dataset.
3. Generating teacher responses.
4. Distilling a smaller student model.

All steps run on small models so the demo fits in limited memory.


## 1. Install dependencies

The demo relies on common libraries from the `test7` project. The cell below installs the minimal set required for a CPU run.

In [ ]:
!pip -q install torch transformers datasets

## 2. Imports and utility setup

In [ ]:
import json, math, requests
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM


## 3. Fetch recent K-line data

The code below is adapted from `src/data/eastmoney_client.py` and `scripts/fetch_eastmoney.py` to retrieve recent daily K-line data for a given stock code.

In [ ]:
def to_secid(symbol: str) -> str:
    symbol = symbol.upper()
    if symbol.startswith(('SH', 'SZ')):
        code = symbol[-6:]
        prefix = symbol[:2]
    else:
        code = symbol[-6:]
        prefix = 'SH' if code[0] in {'5','6','9'} else 'SZ'
    exch = '1' if prefix == 'SH' else '0'
    return f"{exch}.{code}"

def fetch_kline(secid: str, beg: str, end: str,
                fields1: str="f1,f2,f3,f4,f5,f6",
                fields2: str="f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f116"):
    url = "https://push2his.eastmoney.com/api/qt/stock/kline/get"
    params = {
        'secid': secid,
        'beg': beg,
        'end': end,
        'klt': 101,
        'fqt': 1,
        'fields1': fields1,
        'fields2': fields2,
    }
    resp = requests.get(url, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

def parse_kline_json(raw):
    klines = raw.get('data', {}).get('klines', [])
    rows = []
    for item in klines:
        parts = item.split(',')
        rows.append({
            'date': parts[0],
            'open': float(parts[1]),
            'close': float(parts[2]),
            'high': float(parts[3]),
            'low': float(parts[4]),
            'volume': float(parts[5]),
            'turnover': float(parts[6])
        })
    return rows

def get_recent_kline(symbol: str, days: int = 30):
    secid = to_secid(symbol)
    end = pd.Timestamp.today().strftime('%Y%m%d')
    beg = (pd.Timestamp.today() - pd.Timedelta(days=days*2)).strftime('%Y%m%d')
    raw = fetch_kline(secid, beg, end)
    rows = parse_kline_json(raw)
    df = pd.DataFrame(rows)
    if not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df.sort_values('date', inplace=True)
        df = df.tail(days)
    return df

symbol = '600519'
kline_df = get_recent_kline(symbol, days=30)
print(kline_df.head())


## 4. Build prompt sample

We convert the K-line table into a text prompt that asks the model to provide a prediction, analysis and advice. The utility mirrors `build_prompts_from_kline` from the project.

In [ ]:
TEMPLATE = (
    "股票 {stock_code} 近30日K线数据: {kline_json}
"
    "涨跌幅: {change}%。请预测后市走势，给出简短分析和操作建议，"
    "并以 JSON 格式回复，包括 'prediction', 'analysis', 'advice' 三个字段。"
)

def build_prompt_from_df(df: pd.DataFrame, stock_code: str) -> str:
    df = df.copy()
    change = ((df['close'].iloc[-1] / df['close'].iloc[0]) - 1) * 100
    records = df[['date','open','close','high','low','volume']]
    records['date'] = records['date'].dt.strftime('%Y-%m-%d')
    kline_json = records.to_dict(orient='records')
    return TEMPLATE.format(stock_code=stock_code, kline_json=json.dumps(kline_json, ensure_ascii=False), change=round(change,2))

prompt = build_prompt_from_df(kline_df, symbol)
print(prompt[:200] + '...')


## 5. Teacher model generation

For demonstration we use the lightweight `distilgpt2` model as the teacher. It produces a JSON-like answer for the prompt above.

In [ ]:
teacher_model = AutoModelForCausalLM.from_pretrained('distilgpt2')
teacher_tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

inputs = teacher_tokenizer(prompt, return_tensors='pt')
with torch.no_grad():
    teacher_output = teacher_model.generate(**inputs, max_new_tokens=64)

teacher_text = teacher_tokenizer.decode(teacher_output[0], skip_special_tokens=True)
print(teacher_text)


## 6. Distill to a tiny student

We fine-tune a much smaller `sshleifer/tiny-gpt2` model so that its logits match the teacher's on the same prompt. This captures the essence of logits-based knowledge distillation.

In [ ]:
student_model = AutoModelForCausalLM.from_pretrained('sshleifer/tiny-gpt2')
student_tokenizer = AutoTokenizer.from_pretrained('sshleifer/tiny-gpt2')

# Use teacher tokenizer to align vocab
inputs = teacher_tokenizer(prompt, return_tensors='pt')
with torch.no_grad():
    teacher_logits = teacher_model(**inputs).logits

optimizer = torch.optim.AdamW(student_model.parameters(), lr=1e-3)
T = 2.0  # temperature
for step in range(100):
    student_logits = student_model(**inputs).logits
    loss = F.kl_div(
        F.log_softmax(student_logits / T, dim=-1),
        F.softmax(teacher_logits / T, dim=-1),
        reduction='batchmean'
    ) * (T**2)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if step % 20 == 0:
        print(f"step {step}: loss={loss.item():.4f}")


## 7. Student model output

After distillation, the student generates its own response to the same prompt.

In [ ]:
student_output = student_model.generate(**inputs, max_new_tokens=64)
student_text = teacher_tokenizer.decode(student_output[0], skip_special_tokens=True)
print(student_text)


The notebook demonstrated how raw market data can be turned into prompts and how a smaller model can be distilled from a larger one using logits matching—all within a single, self-contained workflow.